In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '..')
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import render
import trajectory
import rooms.dataset
from IPython.display import Audio

# Loading Hallway Data
D = rooms.dataset.dataLoader("espoo_S2")

# Generate Trajectory
indices = [0, 1,2,3,4,5,6]
key_points = D.xyzs[indices]*np.array([1,1,0]) + np.array([0,0,1.01])
tck, u = scipy.interpolate.splprep(key_points.T, k=5)
u_new = np.linspace(0, 1, 100)
interpolated = np.array(scipy.interpolate.splev(u_new, tck)).T
plt.plot(interpolated[:,0], interpolated[:,1], label='Interpolated Trajectory')
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('X (meters)')
plt.ylabel('Y (meters)')
plt.title("Hallway Trajectory")
plt.show()

In [ ]:
# Compute Forward-Facing Vectors for each interpolation point.
forwards = np.diff(interpolated,axis=0)
forwards = forwards/np.linalg.norm(forwards,axis=-1).reshape(-1, 1)

# Compute Left-Facing Vectors
lefts = []
for i in range(forwards.shape[0]):
    lefts.append(np.array([-forwards[i,1], forwards[i,0], 0 ]))
lefts=np.array(lefts)


# There are 99 keypoints - the source is at a constant location for all of them.
source_xyzs = np.array([D.speaker_xyz]*99)


T = trajectory.Trajectory("hallwayTraj", "hallwayBase",interpolated[:99], forwards, lefts, source_xyzs, None, None)
T.source_axis_1s = [None]*99 # There is no source Rotation.
T.source_axis_2s = [None]*99

# Tracing Reflection Paths for all keypoints
T.save_paths()

In [ ]:
# Computing RIRs at all keypoints using one of our trained models.
R = render.Renderer(n_surfaces=6, toa_perturb=False)
R = R.cuda()
T.get_RIRs(R,"../models/hallwayBase/weights.pt")

In [ ]:
T.load_RIRs()

# Convolving RIRs with source audio
T.convolve_audio("GoldenSlumbersMono.wav")

# Smoothly interpolating between audio recordings
rendered_audio = T.fill("GoldenSlumbersMono.wav", fade_length=5)

# Playing Audio
from IPython.display import Audio
Audio(rendered_audio, rate=48000)